In [1]:
from arcgis.gis import GIS
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')

import reach_tools

%load_ext autoreload
%autoreload 2

In [2]:
url_reach_line = os.getenv('URL_REACH_LINE')
url_reach_centroid = os.getenv('URL_REACH_CENTROID')
url_reach_points = os.getenv('URL_REACH_POINT')

reach_id_ldub = 2156
reach_id_farmies = 2269
reach_id_truss = 2270
reach_id_bz = 3064
reach_id_canyon = 3066
reach_id_opal = 5199

lst_reach_id = [reach_id_ldub, reach_id_farmies, reach_id_truss, reach_id_bz, reach_id_canyon, reach_id_opal]

In [3]:
gis = GIS(username=os.getenv('ARCGIS_USERNAME'), password=os.getenv('ARCGIS_PASSWORD'))
gis

GIS @ https://knu2xs.maps.arcgis.com version:6.4

In [4]:
lyr_reach_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_reach_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_reach_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

In [5]:
reach = reach_tools.Reach.get_from_aw(2270)
webmap = reach.update_putin_takeout_and_trace(webmap=True)
webmap

MapView(layout=Layout(height='400px', width='100%'))